# Getting started with Cortex Search Optimization

In [ ]:
-- verify current environment
SELECT 
    CURRENT_WAREHOUSE(),
    CURRENT_DATABASE(),
    CURRENT_SCHEMA(),
    CURRENT_ROLE();

#  Step 1: Role and Warehouse Setup

### Overview
This script configures a basic Snowflake compute environment by:
- Switching to the demoadmin role for administrative access
- Creating a small warehouse optimized for basic workloads

### Configuration Details
The warehouse is configured with:
- Small compute size for cost-effective processing
- 10-minute auto-suspend to optimize credit usage 
- Auto-resume enabled for seamless query execution

In [ ]:
USE ROLE demoadmin;
CREATE WAREHOUSE "MY_WH" 
WITH 
    WAREHOUSE_SIZE = 'SMALL'
    AUTO_SUSPEND = 600
    AUTO_RESUME = TRUE;


# Step 2: Acquiring Data from Snowflake Marketplace

## Data Acquisition from Snowflake Marketplace

### Overview
This section guides through the process of acquiring sample data for search optimization testing. The data comes from Cybersyn's LLM Training dataset, which contains various data types suitable for demonstrating search optimization capabilities.

### Data Source Details
The LLM Training dataset provides:
- Multiple supported data types for optimization
- Real-world data patterns
- Comprehensive test scenarios
- Ready-to-use sample data

### Implementation Steps
1. Access Snowflake Marketplace
2. Locate Cybersyn's LLM Training dataset
3. Complete user profile if needed
4. Create shared database
5. Prepare for data querying

# Step 3: Data Setup
## Database and Schema Setup

### Overview
This script initializes the database environment for search optimization by:
- Setting appropriate role and warehouse context
- Creating a new database for LLM training
- Displaying available schemas

### Configuration Details
The setup includes:
- Using demoadmin role with required privileges
- Using an existing warehouse for compute resources
- Creating a fresh database named LLM_TRAINING_SO
- Verifying schema configuration

In [ ]:
//Note: Use appropriate role having required privileges as mentioned 
//https://docs.snowflake.com/en/user-guide/search-optimization/enabling#required-access-control-privileges
USE ROLE demoadmin;

//Note: Substitute warehouse name my_wh with your warehouse name if different
USE WAREHOUSE my_wh;

//Note: Substitute database name so_analysis with your choice of database name(if required)
CREATE OR REPLACE DATABASE LLM_TRAINING_SO;

USE DATABASE LLM_TRAINING_SO;

SHOW SCHEMAS;

## Schema Creation and Warehouse Scaling

### Overview
This script performs two key operations:
- Creates and sets up a custom schema for data organization
- Scales up the warehouse compute capacity for improved performance

### Configuration Details
The setup includes:
- Creating a new CYBERSYN schema within the current database
- Upgrading warehouse size to 4X-Large for handling larger workloads
- Setting appropriate schema context for subsequent operations

In [ ]:


//Note: Create Schema name of your choice if you do not want to use PUBLIC schema
USE SCHEMA public;
CREATE SCHEMA CYBERSYN;
USE SCHEMA CYBERSYN;

//Note: Substitute my_wh with your warehouse name if different and use warehouse size of your choice
ALTER WAREHOUSE my_wh set warehouse_size='4x-large';

## Data Import Setup

### Overview
This script creates a new table by copying data from an existing OpenAlex works index. 
The operation involves:
- Creating a complete copy of the OpenAlex works index
- Performance depends on warehouse compute capacity
- Data will be used for search optimization testing

### Performance Note
The execution time for this data copy operation varies based on:
- Configured warehouse size
- Volume of data being copied
- Current system load

In [ ]:


//This query time will depend on the warehouse size.
CREATE OR REPLACE TABLE OPENALEX_WORKS_INDEX AS SELECT * FROM LLM_TRAINING.CYBERSYN.OPENALEX_WORKS_INDEX;

## Table Structure Analysis

### Overview
This script examines the structure of the OpenAlex works index table, displaying:
- Column names and their data types
- Constraints and default values
- Nullability of columns
- Any additional column properties or comments

In [ ]:
//Note: Check the table details
DESCRIBE TABLE OPENALEX_WORKS_INDEX;


## Table Verification and Status Check

### Overview
This script displays detailed information about the OpenAlex works index table, including:
- Table existence in the specified schema
- Creation timestamp
- Table size and row count
- Owner and retention period
- Additional table properties and parameters

In [ ]:

SHOW TABLES LIKE 'OPENALEX_WORKS_INDEX' IN SCHEMA LLM_TRAINING_SO.CYBERSYN;

## Table DDL Inspection

### Overview
This script retrieves the complete Data Definition Language (DDL) statement for the OpenAlex works index table, showing:
- Full table creation syntax
- Column definitions and data types
- Table constraints and properties
- Storage parameters and clustering information

### Purpose
Examining the DDL helps in:
- Understanding the complete table structure
- Verifying table configuration details
- Documenting table specifications
- Planning any necessary modifications

In [ ]:

//Note: Check the table details by looking at the table DDL.
SELECT GET_DDL('table','LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX');

## Data Sample Inspection

### Overview
This script retrieves a sample of records from the OpenAlex works index table to:
- Preview actual data content
- Verify data quality and format
- Understand the structure of stored information
- Confirm successful data loading

### Purpose
Examining sample data helps in:
- Validating data import accuracy
- Understanding data patterns
- Planning query optimization strategies
- Identifying potential data quality issues



In [ ]:

//Note: Check the data (Optional)
SELECT * FROM LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX LIMIT 100;

## Cache and Warehouse Configuration Reset

### Overview
This script prepares the environment for accurate performance testing by:
- Disabling query result caching
- Adjusting warehouse size
- Clearing warehouse-level cache

### Purpose
These configuration changes ensure:
- Clean performance benchmarking
- Accurate query execution times
- Consistent test environment
- Reliable comparison metrics before enabling search optimization

In [ ]:
// Note: Optional to execute the queries before enabling Search Optimization on the table

ALTER SESSION SET use_cached_result = false; -- to clear cached data

ALTER WAREHOUSE my_wh SET warehouse_size = MEDIUM;

ALTER WAREHOUSE my_wh SUSPEND; -- to clear data cached at the warehouse level



In [ ]:
SHOW WAREHOUSES;

## Warehouse Size Optimization

### Overview
This script modifies warehouse configuration to:
- Resume warehouse operations
- Scale down compute resources for cost efficiency
- Prepare for normal workload operations

### Purpose
These adjustments help in:
- Managing compute costs
- Right-sizing resources for standard operations
- Ensuring appropriate resource allocation
- Transitioning from testing to production settings

In [ ]:
-- ALTER WAREHOUSE my_wh RESUME;

ALTER WAREHOUSE my_wh SET warehouse_size= 'X-SMALL';

## Single Record Lookup Performance Test

### Overview
This script performs a targeted query to:
- Retrieve a specific record using mag_work_id
- Demonstrate baseline query performance
- Test non-optimized search behavior
- Establish performance benchmark

### Performance Note
This query typically:
- Takes approximately 2 minutes to execute
- Performs a full table scan
- Serves as a baseline for comparing optimized vs non-optimized queries
- Will be used to demonstrate search optimization benefits

In [ ]:
// Note: This query will take ~2 minutes 
SELECT *  from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX where mag_work_id = 2240388798; 


## Text Search Performance Test

### Overview
This script executes a case-insensitive title search to:
- Demonstrate text-based search performance
- Test non-optimized string matching
- Establish baseline for text search operations
- Compare with future search optimization improvements

### Performance Note
This query characteristics:
- Approximately 2.5 minutes execution time
- Uses case-insensitive (ILIKE) pattern matching
- Performs full table scan for text search
- Serves as benchmark for text search optimization

In [ ]:
// Note: This query will take ~2.5 minutes 
SELECT *  from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX where work_title ilike 'Cross-domain applications of multimodal human-computer interfaces'; 


## JSON Path Search Performance Test

### Overview
This script performs a complex search operation to:
- Search within a JSON structure using path notation
- Test case-insensitive matching on nested JSON fields
- Demonstrate baseline performance for semi-structured data queries
- Establish benchmark for JSON search optimization

### Performance Note
Query characteristics include:
- Approximately 3 minutes execution time
- Searches nested JSON path using ILIKE
- Performs full table scan on semi-structured data
- Will be used to compare with optimized JSON search performance

In [ ]:
// Note: This query will take ~3 minutes 
SELECT * from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX  where WORK_PRIMARY_LOCATION:source:display_name ilike 'Eco-forum'; 


## JSON Field Equality Search Performance Test

### Overview
This script tests equality search performance on:
- Nested JSON field using path notation
- Exact matching on ISSN-L identifier
- Deep JSON structure traversal
- Non-optimized variant column search

### Performance Note
Query characteristics include:
- Approximately 4 minutes execution time
- Performs exact matching on JSON path
- Requires full table scan for variant data
- Demonstrates baseline performance for JSON equality searches

In [ ]:
// Note: This query will take ~4 minutes 
SELECT * from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX  where WORK_PRIMARY_LOCATION:source:issn_l = '2615-6946'; 

## Search Optimization Cost Estimation

### Overview
This script estimates the resource costs for implementing search optimization on:
- Exact matching for MAG_WORK_ID
- Case-sensitive matching for source display name in JSON
- Substring matching for work titles
- Substring matching for ISSN-L in JSON structure

### Purpose
The estimation helps in:
- Planning resource allocation
- Understanding storage impact
- Evaluating maintenance costs
- Making informed optimization decisions

In [ ]:
//Note: Optional but recommended step

SELECT SYSTEM$ESTIMATE_SEARCH_OPTIMIZATION_COSTS('LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX',
                                               'EQUALITY(MAG_WORK_ID),EQUALITY(WORK_PRIMARY_LOCATION:source.display_name),
                                               SUBSTRING(WORK_TITLE),SUBSTRING(WORK_PRIMARY_LOCATION:source.issn_l)')
AS estimate_for_columns_without_search_optimization;

## Search Optimization Implementation

### Overview
This script implements various search optimization strategies by:
- Adding equality optimization for numeric IDs
- Enabling substring search for title text
- Optimizing JSON field searches
- Verifying optimization settings

### Optimization Types
The implementation includes:
1. Numeric field optimization for exact matches
2. Text field optimization for partial matches
3. JSON field optimization for both exact and partial matches
4. Table verification after optimization

In [ ]:
// Defining Search Optimization on NUMBER fields For Equality
ALTER TABLE LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX ADD SEARCH OPTIMIZATION ON EQUALITY(MAG_WORK_ID);

// Defining Search Optimization on VARCHAR fields optimized for Wildcard search
ALTER TABLE LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX ADD SEARCH OPTIMIZATION ON SUBSTRING(WORK_TITLE);

// Defining Search Optimization on VARIANT field For Equality
ALTER TABLE LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX ADD SEARCH OPTIMIZATION ON SUBSTRING(WORK_PRIMARY_LOCATION:source.display_name);

// Defining Search Optimization on VARIANT field For Wildcard search
ALTER TABLE LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX ADD SEARCH OPTIMIZATION ON EQUALITY(WORK_PRIMARY_LOCATION:source.issn_l);

SHOW TABLES LIKE 'OPENALEX_WORKS_INDEX' IN SCHEMA LLM_TRAINING_SO.CYBERSYN;

## Search Optimization Status Verification

### Overview
This script inspects the search optimization configuration to:
- Verify optimization status for each configured column
- Check if indexing process is complete
- Confirm active status of optimization methods
- Display detailed optimization settings

### Expected Results
The query returns:
- Expression IDs for each optimization
- Search methods (EQUALITY/SUBSTRING)
- Target columns and their data types
- Active status indicators

In [ ]:
DESCRIBE SEARCH OPTIMIZATION ON LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX;


# Step 4: Equality and Wildcard Search
## Optimized ID Search Performance Test

### Overview
This script tests the performance of:
- Exact matching on mag_work_id after optimization
- Single record retrieval efficiency
- Search optimization effectiveness for numeric fields
- Query execution time improvement

### Purpose
The query demonstrates:
- Performance gains from equality search optimization
- Reduced partition scanning
- Improved response time for exact matches
- Optimization effectiveness for numeric fields

In [ ]:
SELECT *  
  FROM LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX 
  WHERE 
    mag_work_id = 2240388798;

## Optimized Title Search Performance Test

### Overview
This script evaluates:
- Case-insensitive title search performance after optimization
- Substring search optimization effectiveness
- Query execution improvement for text fields
- Performance gains for ILIKE operations

### Purpose
The query demonstrates:
- Optimized text search capabilities
- Performance improvement for case-insensitive matches
- Effectiveness of substring search optimization
- Reduced execution time compared to pre-optimization baseline

In [ ]:
SELECT *  
  FROM LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX 
  WHERE 
    work_title ilike 'Cross-domain applications of multimodal human-computer interfaces'; 

# Step 5: Searching in Variant data
## Optimized JSON Field Search Performance Test

### Overview
This script evaluates:
- Exact matching on JSON field after optimization
- Search performance for nested JSON paths
- Optimization effectiveness for variant columns
- Query execution improvement for ISSN-L searches

### Purpose
The query demonstrates:
- Performance gains for JSON field searches
- Reduced data scanning requirements
- Improved execution time from ~4 minutes to seconds
- Effectiveness of equality optimization on variant data

In [ ]:
select * 
  from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX  
  where 
    WORK_PRIMARY_LOCATION:source:issn_l = '2615-6946';

## Optimized JSON Text Search Performance Test

### Overview
This script evaluates:
- Case-insensitive search in JSON fields after optimization
- Performance improvement for ILIKE operations on nested data
- Search optimization effectiveness for variant columns
- Query execution improvement for display name searches

### Purpose
The query demonstrates:
- Performance gains from substring search optimization on JSON
- Reduced execution time from ~3 minutes to seconds
- Improved efficiency for case-insensitive pattern matching
- Effectiveness of JSON path optimization

In [ ]:
select * 
  from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX  
  where 
    WORK_PRIMARY_LOCATION:source:display_name ilike 'Eco-forum'; 

# Step 6: Queries that are not benefitting from Search Optimization
## Optimized Publication Search Performance Test

### Overview
This script evaluates:
- Case-insensitive search for publication names in JSON
- Performance of optimized variant column searches
- Pattern matching efficiency for source names
- Search optimization impact on JSON path queries

### Purpose
The query demonstrates:
- Improved performance for publication name searches
- Efficient handling of ILIKE operations on JSON fields
- Reduced data scanning through optimization
- Enhanced query response time for variant columns

In [ ]:
select * 
  from LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX  
  where 
    WORK_PRIMARY_LOCATION:source:display_name ilike 'Reactions Weekly'; 

# Cleanup

In [ ]:
-- Use the same role
USE ROLE demoadmin;

-- First remove search optimizations from the table
ALTER TABLE IF EXISTS LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX DROP SEARCH OPTIMIZATION ON EQUALITY(MAG_WORK_ID);
ALTER TABLE IF EXISTS LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX DROP SEARCH OPTIMIZATION ON SUBSTRING(WORK_TITLE);
ALTER TABLE IF EXISTS LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX DROP SEARCH OPTIMIZATION ON SUBSTRING(WORK_PRIMARY_LOCATION:source.display_name);
ALTER TABLE IF EXISTS LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX DROP SEARCH OPTIMIZATION ON EQUALITY(WORK_PRIMARY_LOCATION:source.issn_l);

In [ ]:

-- Drop the table
DROP TABLE IF EXISTS LLM_TRAINING_SO.CYBERSYN.OPENALEX_WORKS_INDEX;

-- Drop the schema
DROP SCHEMA IF EXISTS LLM_TRAINING_SO.CYBERSYN;

-- Drop the database
DROP DATABASE IF EXISTS LLM_TRAINING_SO;

In [ ]:
use warehouse compute_wh;
show warehouses;

In [ ]:


-- Suspend and drop the warehouse
ALTER WAREHOUSE IF EXISTS my_wh SUSPEND;
DROP WAREHOUSE IF EXISTS my_wh;

In [ ]:



-- Reset session parameter
ALTER SESSION SET use_cached_result = true;